In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import sys
import QuantLib as ql

sys.path.insert(0, "/home/christophe/gitlab/norm/test/scripts/python")
sys.path.insert(0, "/home/christophe/gitlab/norm/release/api/python/ct")
sys.path.insert(0, "/home/christophe/gitlab/norm/test/scripts/python/Unitaire")

import import_allocation as ia
import import_pricing as pr
import TestWREmodelingCovFiltering as cf

In [2]:
#  VLVY FRN 11/25 EMTN
#      "date" : "2024-12-19",
#      "close" : 100.288,
#      "ask" : 100.325,
#      "bid" : 100.25,
#       "accIntPrice" : 0.32660833
#      "issueDate" : "2023-11-21",
#    "maturityDate" : "2025-11-21",
#       "marginAdd" : 0.55,
#         "_id" : "251374", ==> euribor 3M
#    "collateralType" : "COMPANY GUARNT",
#     "emtn" : true, euro medium term note
#      "nextCouponDate" : "2025-02-21",
#      "prevCouponDate" : "2024-11-21",
#      "firstCouponDate" : "2024-02-21"

In [3]:
#Données
quotationAMUNDI=100.288
accIntPriceAMUNDI= 0.32660833
year=365.
today = ql.Date(19, ql.December, 2024) #date de calcul
margin_add = 0.55/100
nominal=100

ibordatesnp= np.array([0,7,30,90,180,365, 504, 730, 1095, 1460,1825,2190])
ibordates=ibordatesnp.tolist()
iborvaleursnp= np.array([2.916,2.895,2.795,2.839,2.637, 2.29899, 2.161986, 2.116, 2.08, 2.129, 2.183, 2.2])/100
iborvaleurs=iborvaleursnp.tolist()
ibordatesnp= ibordatesnp.astype(float)/year

start_date = ql.Date(21, ql.November, 2023)
end_date = start_date + ql.Period(2, ql.Years) 

#float
coupon_tenor_year=0.25 # year
last_coupon_date =  ql.Date(21, ql.November, 2024)
nbcoupon_fl = 4

In [4]:
ibordates

[0, 7, 30, 90, 180, 365, 504, 730, 1095, 1460, 1825, 2190]

In [5]:
len(iborvaleurs)

12

In [6]:
ibordatesnp

array([0.        , 0.01917808, 0.08219178, 0.24657534, 0.49315068,
       1.        , 1.38082192, 2.        , 3.        , 4.        ,
       5.        , 6.        ])

In [7]:
### quantlib

In [8]:
ql.Settings.instance().evaluationDate = today

In [9]:
def nparray2listQlDates(today,dates, unit):
    qldates = []
    for i in range(len(dates)):
        if dates[i]>=0:
            qldates.append(today+ql.Period(dates[i], unit))
        else:
            print(- dates[i])
            qldates.append(today-ql.Period(-dates[i], unit))
    return qldates

In [10]:
ibordatesql = nparray2listQlDates(today,ibordates, ql.Days) #ibordates est en jour (quantlib), ibordatesnp est en année (wre)

In [11]:
ibordatesql

[Date(19,12,2024),
 Date(26,12,2024),
 Date(18,1,2025),
 Date(19,3,2025),
 Date(17,6,2025),
 Date(19,12,2025),
 Date(7,5,2026),
 Date(19,12,2026),
 Date(19,12,2027),
 Date(18,12,2028),
 Date(18,12,2029),
 Date(18,12,2030)]

In [12]:
emtn_curve = ql.ZeroCurve(
    ibordatesql,
    iborvaleurs,
    ql.Actual365Fixed()
)
ncurve = len(ibordates)

In [13]:
emtn_handle = ql.YieldTermStructureHandle(emtn_curve)

In [14]:
emtn = ql.Euribor(
    ql.Period(3, ql.Months), emtn_handle
)

In [15]:
d = start_date-30
while d <= today:
    if emtn.isValidFixingDate(d):
        emtn.addFixing(d, iborvaleurs[0])
    d += 1

In [16]:
rule = ql.DateGeneration.Forward
end_of_month = False
#coupon_tenor = ql.Period(coupon_tenor_year, ql.Years)
settlement_days = 0
fixing_days = 0
schedule = ql.Schedule( start_date,
                        end_date,
                        ql.Period('3M'),
                        ql.NullCalendar(),
                        ql.Unadjusted, 
                        ql.Unadjusted,
                        rule, 
                        False)
frn = ql.FloatingRateBond(settlement_days,
                             nominal,
                             schedule,
                             emtn,
                             ql.Thirty360(ql.Thirty360.BondBasis),
                             ql.Unadjusted,
                             fixing_days,
                             [],      # Gearings
                             [margin_add],     # Spreads
                             [],      # Caps
                             [],      # Floors
                             True,   # Fixing in arrears
                             nominal,
                             start_date)


In [17]:
start_date

Date(21,11,2023)

In [18]:
end_date

Date(21,11,2025)

In [19]:
# coupon pricers  ???
pricer = ql.BlackIborCouponPricer()
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
volatility = 0.0
vol = ql.ConstantOptionletVolatility(settlement_days,
                                  calendar,
                                  ql.Unadjusted,
                                  volatility,
                                  ql.Thirty360(ql.Thirty360.BondBasis))

pricer.setCapletVolatility(ql.OptionletVolatilityStructureHandle(vol))
ql.setCouponPricer(frn.cashflows(), pricer)

In [20]:
frn.setPricingEngine(
    ql.DiscountingBondEngine(emtn_handle) # estr_handle, euribor6m_handle
)

In [21]:
frn.NPV()

100.47153204009653

In [22]:
frn.cleanPrice()

100.22995444929869

In [23]:
frn.dirtyPrice()

100.47153204009653

In [24]:
cfs = frn.cashflows()
coupons = [ ql.as_coupon(c) for c in cfs[:-1] ] # the last one is the redemption
print( [ (c.rate(), c.accrualPeriod()) for c in coupons ])

[(0.034659999999999996, 0.25), (0.034659999999999996, 0.25), (0.034659999999999996, 0.25), (0.034659999999999996, 0.25), (0.031059975959723148, 0.25), (0.0273646520846399, 0.25), (0.02416472472002209, 0.25), (0.0235019435100318, 0.25)]


In [25]:
data = []
for cf in frn.cashflows():
    coupon = ql.as_coupon(cf)
    print(coupon)
    data.append(
        (
            #coupon.rate(),
            #coupon.accrualPeriod(),
            #coupon.amount(),
        )
    )

<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ed407c330> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ed407c3f0> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ef59fc540> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ed556f540> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ef59fc540> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ed556f540> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ef59fc540> >
<QuantLib.QuantLib.Coupon; proxy of <Swig Object of type 'ext::shared_ptr< Coupon > *' at 0x761ed556f540> >
None


In [26]:
#a=pd.DataFrame(
#    data, columns=["date", "rate", "tenor", "amount"]
#).style.format(
#    {"amount": "{:.2f}", "rate": "{:.2%}"}
#)

In [27]:
# WRE
pricing = pr.Pricing(pathlib='/home/christophe/gitlab/norm/release/lib/linux64/')

Loading Pricing module:
Pricing module loaded.


In [28]:
# pate flottante floatrate :
quarteryear=93.
decal = today-last_coupon_date
mfl=nbcoupon_fl
t1fl=np.zeros(mfl,dtype='float')
t2fl=np.zeros(mfl,dtype='float')
for i in range(mfl):
    t2fl[i]=((i+1)*quarteryear-decal)/year #dates of coupons  ## a generaliser !!
info = np.zeros(1,dtype='int64')
actratfl=np.zeros(mfl)
couponfl=np.zeros(mfl)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, ibordatesnp.astype(float), iborvaleursnp.astype(float), mfl, t1fl.astype(float), t2fl.astype(float), interpmethod, actratfl, info )
t1fl[1:]=t2fl[0:-1]
pricing.WREpricingCurve2Forward( ncurve, ibordatesnp.astype(float), iborvaleursnp.astype(float), mfl, t1fl.astype(float), t2fl.astype(float), interpmethod, couponfl, info )


 r1=   2.9159999999999998E-002   2.9159999999999998E-002   2.9159999999999998E-002   2.9159999999999998E-002
 r2=   2.8206666666666665E-002   2.6863777777777778E-002   2.5072772432432430E-002   2.3373587027027024E-002
 f12=   2.8206666666666665E-002   2.6863777777777778E-002   2.5072772432432430E-002   2.3373587027027024E-002
 r1=   2.9159999999999998E-002   2.8206666666666665E-002   2.6863777777777778E-002   2.5072772432432430E-002
 r2=   2.8206666666666665E-002   2.6863777777777778E-002   2.5072772432432430E-002   2.3373587027027024E-002
 f12=   2.8206666666666665E-002   2.5925199522102753E-002   2.2029989157544633E-002   1.8787613513513515E-002


In [29]:
t1fl

array([0.        , 0.17808219, 0.43287671, 0.68767123])

In [30]:
t2fl

array([0.17808219, 0.43287671, 0.68767123, 0.94246575])

In [31]:
t1fl

array([0.        , 0.17808219, 0.43287671, 0.68767123])

In [32]:
quarteryear

93.0

In [33]:
decal

28

In [34]:
# jambe flottante
ndatesFL = mfl
datesFL = t2fl
actualizationratesFL = actratfl
principalFL = 100.0
yearfractionFL = 0.25
floatingrate = couponfl


#valo
# PRICEFRN ( ndates, dates, actualizationrates, forwardfloatingrate, spreadc, yearfraction, &
#                      Principal, Principalexchanged, floorr, cap, autoactualized, Pfrn, info)
priceFRN = np.zeros(1)
pricing.WREpricingFRN( ndatesFL,  datesFL,  actualizationratesFL,  floatingrate ,  margin_add, yearfractionFL, principalFL,
                      1, 0.0, 1.0, 0, priceFRN, info)

 principal actualized from floating leg=   97.821205694611166     
 test   100.00000000000000        2.3373587027027024E-002  0.94246575342465755                4
 floatingleg=   100.70664288818880     
 Pfrn=   100.70664288818880     


In [35]:
###### Comparaison

In [36]:
data={"Market Value": [quotationAMUNDI+accIntPriceAMUNDI, quotationAMUNDI],
      "WRE":[priceFRN[0],"x"],
      "Quantlib": [frn.NPV(),frn.cleanPrice()], 
     }

pd.DataFrame(data, index=['Dirty Price', 'Clean Price'])


,Market Value,WRE,Quantlib
Dirty Price,100.614608,100.706643,100.471532
Clean Price,100.288000,x,100.229954
